In [1]:
from openff.toolkit import ForceField, Molecule, Topology
from openmm import LangevinMiddleIntegrator, unit
from openff.interchange import Interchange
import mdtraj

In [2]:
integrator = LangevinMiddleIntegrator(300, 1, 4)

In [3]:
print("Building ligand..")
ligand = Molecule.from_file("final.sdf", allow_undefined_stereo=True)

Building ligand..


In [4]:
ligand_intrcg = Interchange.from_smirnoff(force_field=ForceField("openff_unconstrained-2.0.0.offxml"), topology=[ligand],)

In [5]:
print("\nBuilding protein..")
protein = Topology.from_pdb("final.pdb") # apo protein + ions


Building protein..


In [6]:
protein_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=protein,
)

In [7]:
%env INTERCHANGE_EXPERIMENTAL=1

env: INTERCHANGE_EXPERIMENTAL=1


In [8]:
print("\nBuilding complex..")
complex_intrcg = protein_intrcg + ligand_intrcg


Building complex..


/lila/home/takabak/mambaforge/envs/espfit/lib/python3.11/site-packages/openff/interchange/_experimental.py:35: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  return func(*args, **kwargs)


In [9]:
print("Minimizing..")
simulation = complex_intrcg.to_openmm_simulation(integrator)
start_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
simulation.minimizeEnergy(maxIterations=3)
end_pot_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

Minimizing..


In [10]:
print(f"{round(start_pot_energy, 2)} -> {round(end_pot_energy, 2)}")

1259.43 -> -3718.81


In [11]:
# check complex topology
res = [ r.name for r in simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'UNK',
 'VAL'}

#### (Approach 1) DEBUG: Compute potential energy by reloading minimized protein structure

In [12]:
simulation.topology

<Topology; 3 chains, 613 residues, 9424 atoms, 9535 bonds>

In [13]:
from openmm.app import PDBFile
with open("min.pdb", "w") as wf:
    PDBFile.writeFile(
        simulation.topology,
        simulation.context.getState(
            getPositions=True,
            enforcePeriodicBox=False).getPositions(),
            #enforcePeriodicBox=True).getPositions(),
            file=wf,
            keepIds=True
    )

In [14]:
t = mdtraj.load('min.pdb')
print(t.n_residues)

613


In [15]:
atom_indices = t.topology.select('not resname UNK')
t = t.atom_slice(atom_indices)
print(t.n_residues)

612


In [16]:
t.save_pdb('protein_min.pdb')

In [17]:
# create new system and compute potential energy
import copy
_integrator = copy.deepcopy(integrator)

_protein = Topology.from_pdb("protein_min.pdb") # apo protein + ions
_protein_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=_protein,
)
_simulation = _protein_intrcg.to_openmm_simulation(_integrator)
energy_1 = _simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

print(f"{round(energy_1, 2)}")

-3813.43


In [18]:
del _integrator, _protein, _protein_intrcg, _simulation

#### (Approach 2) Update protein positions and create new system

In [19]:
res = [ r.residue_name for r in protein.residues ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL'}

In [20]:
import copy
_protein = copy.deepcopy(protein)

In [21]:
# initial positions
_protein.get_positions()

Magnitude,[[0.6899000000000001 10.523499999999999 -0.9576] [0.6973999999999999 10.636400000000002 -0.8615999999999999] [0.6038999999999999 10.616500000000002 -0.7406999999999998] ... [-1.0171 8.6242 1.4023999999999999] [-1.2927 8.598600000000001 1.6672000000000002] [-1.1286 8.672699999999999 1.6589999999999996]]
Units,nanometer


In [22]:
# openmm topology
protein_topology = _protein.to_openmm()
# number of protein atoms
n_protein_atoms = protein_topology.getNumAtoms()
# complex position (OpenMM Quantity -> need to convert this to OpenFF Quantity later)
complex_positions = simulation.context.getState(getPositions=True).getPositions()

In [23]:
# get minimized protein positions
import numpy as np
protein_positions = unit.Quantity(np.zeros([n_protein_atoms, 3]), unit=unit.nanometers)
protein_positions = complex_positions[:n_protein_atoms]

In [24]:
# we need to pass OpenFF Quantity
from openff.units.openmm import from_openmm
_protein.set_positions(from_openmm(protein_positions))

In [25]:
# check updated positions
_protein.get_positions()

Magnitude,[[0.6894798010545997 10.52299716515563 -0.9549311329333138] [0.6993675697263687 10.63525237864174 -0.8620148625707426] [0.6034467912870997 10.615788275520227 -0.7451257297330044] ... [-1.0171374265219832 8.624749171345854 1.4036386169858386] [-1.2920328049269487 8.599665488966888 1.6680997699464142] [-1.1303801606255057 8.67334223094333 1.6607076825287554]]
Units,nanometer


In [26]:
# create new system and compute potential energy
import copy
_integrator = copy.deepcopy(integrator)

_protein_intrcg = Interchange.from_smirnoff(
    force_field=ForceField("ff14sb_off_impropers_0.0.3.offxml", "openff_unconstrained-2.0.0.offxml"),
    topology=_protein,
)
_simulation = _protein_intrcg.to_openmm_simulation(_integrator)
energy_2 = _simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

In [27]:
print(f"{round(energy_2, 2)}")

-3814.43


In [28]:
# check topology
res = [ r.name for r in _simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'VAL'}

In [29]:
del _integrator, _protein, _protein_intrcg, _simulation

#### (Approach 3) Turn off ligand FF parameters

In [30]:
res = [ r.name for r in simulation.topology.residues() ]
set(res)

{'ALA',
 'ARG',
 'ASN',
 'ASP',
 'CYS',
 'GLN',
 'GLU',
 'GLY',
 'HIS',
 'ILE',
 'LEU',
 'LYS',
 'MET',
 'PHE',
 'PRO',
 'SER',
 'THR',
 'TRP',
 'TYR',
 'UNK',
 'VAL'}

In [31]:
# ligand atom indices
#resnames = ['UNK', 'NA']
resnames = ['UNK']
ligand_atom_indices = []
for atom in simulation.topology.atoms():
    if atom.residue.name in resnames:
        ligand_atom_indices.append(atom.index)

In [32]:
ligand_atom_indices

[9365,
 9366,
 9367,
 9368,
 9369,
 9370,
 9371,
 9372,
 9373,
 9374,
 9375,
 9376,
 9377,
 9378,
 9379,
 9380,
 9381,
 9382,
 9383,
 9384,
 9385,
 9386,
 9387,
 9388,
 9389,
 9390,
 9391,
 9392,
 9393,
 9394,
 9395,
 9396,
 9397,
 9398,
 9399,
 9400,
 9401,
 9402,
 9403,
 9404,
 9405,
 9406,
 9407,
 9408,
 9409,
 9410,
 9411,
 9412,
 9413,
 9414,
 9415,
 9416,
 9417,
 9418,
 9419,
 9420,
 9421,
 9422,
 9423]

In [33]:
# get forces
forces = list(simulation.system.getForces())
forces

[<openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f2714b78030> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7f2714b79c50> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7f2714b7b330> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7f2714b78c00> >]

In [34]:
def check_ligand_forces(ligand_atom_indices):
    # check ligand forces
    for force in forces:
        name = force.__class__.__name__
    
        if "Bond" in name:
            for idx in range(force.getNumBonds()):
                p1, p2, length, k = force.getBondParameters(idx)
                sublist = [p1, p2]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Bond: {sublist}, {length}, {k}')
    
        elif "Angle" in name:
            for idx in range(force.getNumAngles()):
                p1, p2, p3, angle, k = force.getAngleParameters(idx)
                sublist = [p1, p2, p3]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Angle: {sublist}, {angle}, {k}')
                    
        elif "Torsion" in name:
            for idx in range(force.getNumTorsions()):
                p1, p2, p3, p4, periodicity, phase, k = force.getTorsionParameters(idx)
                sublist = [p1, p2, p3, p4]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Torsion: {sublist}, {periodicity}, {phase}, {k}')
        
        elif "Nonbonded" in name:
            for idx in range(force.getNumParticles()):
                if idx in ligand_atom_indices:
                    q, sigma, epsilon = force.getParticleParameters(idx)
                    print(f'Nonbonded: {idx}, {q}, {sigma}, {epsilon}')
            for idx in range(force.getNumExceptions()):
                p1, p2, q, sigma, epsilon = force.getExceptionParameters(idx)
                sublist = [p1, p2]
                if all(x in ligand_atom_indices for x in sublist):
                    print(f'Nonbonded NumExceptions: {sublist}, {q}, {sigma}, {epsilon}')

In [35]:
# check ligand forces before turning off FF parameters
check_ligand_forces(ligand_atom_indices)

Nonbonded: 9365, 0.07728306779661016 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9366, -0.5629169322033898 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9367, -0.6131169322033898 e, 0.3039812205065809 nm, 0.8795023257036865 kJ/mol
Nonbonded: 9368, 0.6710830677966102 e, 0.34806468869450646 nm, 0.3635030558377792 kJ/mol
Nonbonded: 9369, -0.43481693220338985 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9370, -0.5768169322033898 e, 0.2997159987248637 nm, 0.8764372596155737 kJ/mol
Nonbonded: 9371, -0.04061693220338983 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9372, -0.6630169322033899 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9373, -0.6505169322033898 e, 0.3039812205065809 nm, 0.8795023257036865 kJ/mol
Nonbonded: 9374, -0.10141693220338983 e, 0.337953176162662 nm, 0.45538911611061844 kJ/mol
Nonbonded: 9375, -0.6560169322033899 e, 0.3206876023663901 nm, 0.7016212989374017 kJ/mol
Nonbonded: 9376, -

In [36]:
# check potential energy before turning off FF parameters
_energy = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole
print(round(_energy, 2))

-3718.81


In [37]:
for force in forces:
    name = force.__class__.__name__

    if "Bond" in name:
        for idx in range(force.getNumBonds()):
            p1, p2, length, k = force.getBondParameters(idx)
            sublist = [p1, p2]
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Bond: {sublist}')
                force.setBondParameters(idx, p1, p2, length, 0.0)
        force.updateParametersInContext(simulation.context)

    elif "Angle" in name:
        for idx in range(force.getNumAngles()):
            p1, p2, p3, angle, k = force.getAngleParameters(idx)
            sublist = [p1, p2, p3]      
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Angle: {sublist}')
                force.setAngleParameters(idx, p1, p2, p3, angle, 0.0)
        force.updateParametersInContext(simulation.context)
                
    elif "Torsion" in name:
        for idx in range(force.getNumTorsions()):
            p1, p2, p3, p4, periodicity, phase, k = force.getTorsionParameters(idx)
            sublist = [p1, p2, p3, p4]
            if all(x in ligand_atom_indices for x in sublist):
                print(f'Torsion: {sublist}')
                force.setTorsionParameters(idx, p1, p2, p3, p4, periodicity, phase, 0.0)
        force.updateParametersInContext(simulation.context)
    
    elif "Nonbonded" in name:
        for idx in range(force.getNumParticles()):
            if idx in ligand_atom_indices:
                print(f'Nonbonded: {idx}')
                q, sigma, epsilon = force.getParticleParameters(idx)
                
                # set partial charge q to 1e-10 to avoid following error:
                # OpenMMException: updateParametersInContext: The number of non-excluded exceptions has changed
                force.setParticleParameters(idx, q * 1e-10, 0, 0)
        for idx in range(force.getNumExceptions()):
                print(f'Nonbonded NumExceptions: {idx}')
                p1, p2, q, sigma, epsilon = force.getExceptionParameters(idx)
                sublist = [p1, p2]
                if all(x in ligand_atom_indices for x in sublist):
                    force.setExceptionParameters(idx, p1, p2, q * 1e-10, 0, 0)
        force.updateParametersInContext(simulation.context)

Nonbonded: 9365
Nonbonded: 9366
Nonbonded: 9367
Nonbonded: 9368
Nonbonded: 9369
Nonbonded: 9370
Nonbonded: 9371
Nonbonded: 9372
Nonbonded: 9373
Nonbonded: 9374
Nonbonded: 9375
Nonbonded: 9376
Nonbonded: 9377
Nonbonded: 9378
Nonbonded: 9379
Nonbonded: 9380
Nonbonded: 9381
Nonbonded: 9382
Nonbonded: 9383
Nonbonded: 9384
Nonbonded: 9385
Nonbonded: 9386
Nonbonded: 9387
Nonbonded: 9388
Nonbonded: 9389
Nonbonded: 9390
Nonbonded: 9391
Nonbonded: 9392
Nonbonded: 9393
Nonbonded: 9394
Nonbonded: 9395
Nonbonded: 9396
Nonbonded: 9397
Nonbonded: 9398
Nonbonded: 9399
Nonbonded: 9400
Nonbonded: 9401
Nonbonded: 9402
Nonbonded: 9403
Nonbonded: 9404
Nonbonded: 9405
Nonbonded: 9406
Nonbonded: 9407
Nonbonded: 9408
Nonbonded: 9409
Nonbonded: 9410
Nonbonded: 9411
Nonbonded: 9412
Nonbonded: 9413
Nonbonded: 9414
Nonbonded: 9415
Nonbonded: 9416
Nonbonded: 9417
Nonbonded: 9418
Nonbonded: 9419
Nonbonded: 9420
Nonbonded: 9421
Nonbonded: 9422
Nonbonded: 9423
Nonbonded NumExceptions: 0
Nonbonded NumExceptions: 1
No

In [38]:
# check ligand forces after turning off
check_ligand_forces(ligand_atom_indices)

Nonbonded: 9365, 7.728306779661017e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9366, -5.6291693220338984e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9367, -6.131169322033899e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9368, 6.710830677966102e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9369, -4.3481693220338985e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9370, -5.7681693220338985e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9371, -4.061693220338983e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9372, -6.630169322033899e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9373, -6.505169322033898e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9374, -1.0141693220338983e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9375, -6.560169322033898e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9376, -1.0141693220338983e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9377, 1.0498306779661017e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9378, -5.671693220338983e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9379, -9.041693220338982e-12 e, 0.0 nm, 0.0 kJ/mol
Nonbonded: 9380, 1.677830677966102e-11 e, 0.0 nm, 0.0 kJ/mol
Nonbon

In [39]:
# compute potential energy
energy_3 = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

In [40]:
# energy difference between the two approaches
print(f"Approach 1: {round(energy_1, 2)}")    # probably, precision loss because the protein was saved to pdb and reloaded back to compute the potential energies
print(f"Approach 2: {round(energy_2, 2)}")
print(f"Approach 3: {round(energy_3, 2)}")

Approach 1: -3813.43
Approach 2: -3814.43
Approach 3: -3814.43


##### DEBUG: Turn off all FF parameters to see if the potential energy goes to zero

In [41]:
for force in forces:
    name = force.__class__.__name__

    if "Bond" in name:
        for idx in range(force.getNumBonds()):
            id1, id2, length, k = force.getBondParameters(idx)
            force.setBondParameters(idx, id1, id2, length, 0.0)
        force.updateParametersInContext(simulation.context)

    elif "Angle" in name:
        for idx in range(force.getNumAngles()):
            id1, id2, id3, angle, k = force.getAngleParameters(idx)
            force.setAngleParameters(idx, id1, id2, id3, angle, 0.0)
        force.updateParametersInContext(simulation.context)
                
    elif "Torsion" in name:
        for idx in range(force.getNumTorsions()):
            id1, id2, id3, id4, periodicity, phase, k = force.getTorsionParameters(idx)
            force.setTorsionParameters(idx, id1, id2, id3, id4, periodicity, phase, 0.0)
        force.updateParametersInContext(simulation.context)
    
    elif "Nonbonded" in name:
        for idx in range(force.getNumParticles()):
            q, sigma, epsilon = force.getParticleParameters(idx)            
            # set partial charge q to 1e-10 to avoid following error:
            # OpenMMException: updateParametersInContext: The number of non-excluded exceptions has changed
            force.setParticleParameters(idx, q * 1e-10, 0, 0)
        for idx in range(force.getNumExceptions()):
            idx0, idx1, q, sigma, epsilon = force.getExceptionParameters(idx)
            force.setExceptionParameters(idx, idx0, idx1, q * 1e-10, 0, 0)
        force.updateParametersInContext(simulation.context)


energy_4 = simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

print(round(energy_4, 2))

0.0


#### Approach(4) Use `openmm.openmm.NonbondedForce.addParticleParameterOffset` to compute ele and vdw of (G_complex - G_protein - G_ligand)

In [70]:
try:
    del _simulation, _integrator, forces
except:
    pass

In [72]:
# create new simulation object since we updated the context in previous section
complex_positions = simulation.context.getState(getPositions=True).getPositions()   # minimized coordinate
complex_positions[0]

Quantity(value=Vec3(x=0.6894798010545997, y=10.52299716515563, z=-0.9549311329333138), unit=nanometer)

In [73]:
# current potential energy -> should be zero
simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

2.4263525667305505e-06

In [74]:
_integrator = copy.deepcopy(integrator)
_simulation = complex_intrcg.to_openmm_simulation(_integrator)

In [75]:
_simulation.context.getState(getPositions=True).getPositions()[0]   # this should be initial coordinate

Quantity(value=Vec3(x=0.6899000000000001, y=10.523499999999999, z=-0.9576), unit=nanometer)

In [76]:
# check potential energy -> should be before minimization
_simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

1259.4321832098651

In [77]:
# update coordinate
_simulation.context.setPositions(complex_positions)
# check updated coordinate
_simulation.context.getState(getPositions=True).getPositions()[0]

Quantity(value=Vec3(x=0.6894798010545997, y=10.52299716515563, z=-0.9549311329333138), unit=nanometer)

In [78]:
# check potential energy -> should be after minimization
_simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

-3718.8085233229517

In [79]:
# get atom indices
protein_atom_indices = set([a.index for a in _simulation.topology.atoms() if a.residue.name not in ('UNK')])
ligand_atom_indices = set([a.index for a in _simulation.topology.atoms() if a.residue.name in ('UNK')])

In [80]:
# http://docs.openmm.org/development/api-python/generated/openmm.openmm.NonbondedForce.html#openmm.openmm.NonbondedForce.addParticleParameterOffset
import openmm
forces = list(_simulation.system.getForces())

for idx, force in enumerate(_simulation.system.getForces()):
    if isinstance(force, openmm.openmm.NonbondedForce):
        forces[idx].setForceGroup(0)        
        forces[idx].addGlobalParameter("solute_coulomb_scale", 1)
        forces[idx].addGlobalParameter("solute_lj_scale", 1)
        forces[idx].addGlobalParameter("solvent_coulomb_scale", 1)
        forces[idx].addGlobalParameter("solvent_lj_scale", 1)
        for i in range(forces[idx].getNumParticles()):
            charge, sigma, epsilon = forces[idx].getParticleParameters(i)
            forces[idx].setParticleParameters(i, 0, 0, 0)
            if i in protein_atom_indices:
                forces[idx].addParticleParameterOffset("solute_coulomb_scale", i, charge, 0, 0)
                forces[idx].addParticleParameterOffset("solute_lj_scale", i, 0, sigma, epsilon)
            else:
                forces[idx].addParticleParameterOffset("solvent_coulomb_scale", i, charge, 0, 0)
                forces[idx].addParticleParameterOffset("solvent_lj_scale", i, 0, sigma, epsilon)
        for i in range(forces[idx].getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = forces[idx].getExceptionParameters(i)
            forces[idx].setExceptionParameters(i, p1, p2, 0, 0, 0)
    else:
        forces[idx].setForceGroup(2)

In [81]:
# check global parameters
for i in range(forces[0].getNumGlobalParameters()):
    print(i, forces[0].getGlobalParameterName(i))

0 solute_coulomb_scale
1 solute_lj_scale
2 solvent_coulomb_scale
3 solvent_lj_scale


In [82]:
# check potential energy -> nothing should change yet
_simulation.context.getState(getEnergy=True).getPotentialEnergy() / unit.kilocalories_per_mole

-3718.8085233229517

In [83]:
def energy(solute_coulomb_scale, solute_lj_scale, solvent_coulomb_scale, solvent_lj_scale):
    _simulation.context.setParameter("solute_coulomb_scale", solute_coulomb_scale)
    _simulation.context.setParameter("solute_lj_scale", solute_lj_scale)
    _simulation.context.setParameter("solvent_coulomb_scale", solvent_coulomb_scale)
    _simulation.context.setParameter("solvent_lj_scale", solvent_lj_scale)
    return _simulation.context.getState(getEnergy=True, groups={0}).getPotentialEnergy() / unit.kilocalories_per_mole

In [84]:
total_coulomb = energy(1, 0, 1, 0)
solute_coulomb = energy(1, 0, 0, 0)
solvent_coulomb = energy(0, 0, 1, 0)
total_lj = energy(0, 1, 0, 1)
solute_lj = energy(0, 1, 0, 0)
solvent_lj = energy(0, 0, 0, 1)
print('Coulomb interaction energy:', total_coulomb - solute_coulomb - solvent_coulomb)
print('LJ interaction energy:', total_lj - solute_lj - solvent_lj)

OpenMMException: Called setParameter() with invalid parameter name: solute_coulomb_scale